### 保存数据

    通过Serializer验证完成数据之后，我们需要将这些数据保存到数据库中，此时分为两种情况：
        1.保存为一个新的对象；
        2.更新某个对象的数据；
    
    无论是新建还是更新，都需要先调用is_valid方法进行数据验证，然后调用Serializer的save()方法。其通过是否传入instance参数区分更新或新建对象。instance参数为要更新的对象，而不是Serializer实例。当未传入instance参数时，为新建对象；当传入了instance参数时，为更新对象。
    

##### 提示 - 部分更新
    
    Serializer默认会验证所有字段，当只对部分数据进行更新时，则需要在实例化Serializer时传入partial参数：
        serializer = CommentSerializer(comment, data={'content': 'foo bar'}, partial=True)

    
#### Serializer.save()源码

In [ ]:
    def save(self, **kwargs):
        
        """
        现在无需将主要注意力集中在注释的这部分代码
        
        assert not hasattr(self, 'save_object'), (
            'Serializer `%s.%s` has old-style version 2 `.save_object()` '
            'that is no longer compatible with REST framework 3. '
            'Use the new-style `.create()` and `.update()` methods instead.' %
            (self.__class__.__module__, self.__class__.__name__)
        )

        assert hasattr(self, '_errors'), (
            'You must call `.is_valid()` before calling `.save()`.'
        )

        assert not self.errors, (
            'You cannot call `.save()` on a serializer with invalid data.'
        )

        # Guard against incorrect use of `serializer.save(commit=False)`
        assert 'commit' not in kwargs, (
            "'commit' is not a valid keyword argument to the 'save()' method. "
            "If you need to access data before committing to the database then "
            "inspect 'serializer.validated_data' instead. "
            "You can also pass additional keyword arguments to 'save()' if you "
            "need to set extra attributes on the saved model instance. "
            "For example: 'serializer.save(owner=request.user)'.'"
        )

        assert not hasattr(self, '_data'), (
            "You cannot call `.save()` after accessing `serializer.data`."
            "If you need to access data before committing to the database then "
            "inspect 'serializer.validated_data' instead. "
        )
        
        """
        
        # 将调用save时传入的额外kwargs合并到validated_data中
        validated_data = dict(
            list(self.validated_data.items()) +
            list(kwargs.items())
        )
        
        # 通过是否传入模型实例判断是新建对象还是更新已有对象
        if self.instance is not None:
            self.instance = self.update(self.instance, validated_data)
            assert self.instance is not None, (
                '`update()` did not return an object instance.'
            )
        else:
            self.instance = self.create(validated_data)
            assert self.instance is not None, (
                '`create()` did not return an object instance.'
            )

        return self.instance
    
"""
    从源码中可以看到，在进行保存操作前，会将kwargs参数合并到validated_data中，这意味着可以在调用save方法时额外传入
    保存操作所需要，但在序列化器中没有定义的字段。


"""

#### Serializer - 定义create() / update() 方法

    在Serializer中，并没有实现create以及update方法，因此需要自行定义。
     
    官方文档示例：

In [ ]:
# 假定有一个Comment模型


def create(self, validated_data):
    return Comment.objects.create(**validated_data)


def update(self, instance, validated_data):
    instance.email = validated_data.get('email', instance.email)
    instance.content = validated_data.get('content', instance.content)
    instance.created = validated_data.get('created', instance.created)
    instance.save()
    return instance



# 新建对象
data = None    # 前台提交的数据 - 字典格式
ser = CommentSerializer(data=data)
if ser.is_valid():
    ser.save()

# 更新对象
# 全部更新
# ser = CommentSerializer(instance=com, data=data)

# 部分更新
ser = CommentSerializer(instance=com, data=data, partial=True)
if ser.is_valid():
    ser.save(owner=request.user)  # 可以传入额外的参数



#### ModelSerializer

    ModelSerializer默认实现了简单的create以及update方法，但是当模型中含有关联字段时，则需要重写create以及update方法。
    
    保存时，关联字段指定的是对象，而不是pk值。设计many_to_many字段时，即可以利用set方法一次性添加多个对象组成的列表，也可以使用add方法单个依次添加。
    

    官方文档示例：

In [2]:
class UserSerializer(serializers.ModelSerializer):
    profile = ProfileSerializer()

    class Meta:
        model = User
        fields = ['username', 'email', 'profile']

    def create(self, validated_data):
        profile_data = validated_data.pop('profile')
        user = User.objects.create(**validated_data)
        Profile.objects.create(user=user, **profile_data)
        return user
    
    
    def update(self, instance, validated_data):
        profile_data = validated_data.pop('profile')
        # Unless the application properly enforces that this field is
        # always set, the following could raise a `DoesNotExist`, which
        # would need to be handled.
        profile = instance.profile

        instance.username = validated_data.get('username', instance.username)
        instance.email = validated_data.get('email', instance.email)
        instance.save()

        profile.is_premium_member = profile_data.get(
            'is_premium_member',
            profile.is_premium_member
        )
        profile.has_support_contract = profile_data.get(
            'has_support_contract',
            profile.has_support_contract
         )
        profile.save()

        return instance